In [33]:
import pandas as pd

## Pega Coutries list based on UE publication (Anex A5) 

- Following script will collect [countries list](http://publications.europa.eu/code/en/en-5000500.htm) and process it to csv file that can be used by Pega
- Tested on Anex A5 version 5.4.2023 
- All UTF-8 based encodings, ready to use within Pega class: `PegaPS-Data-Config-Country`

In [34]:
df = pd.read_html("http://publications.europa.eu/code/en/en-5000500.htm", encoding='utf8')

In current publication there are two tables defined in document. First none contains revision history while second one contains list of countries. We are after second one which contains countries and countries code which we import to Pega.

In [35]:
df[1].head()

,Unnamed: 0,Short name,Full name,Country code (1),Capital/ administrative centre,Demonym,Adjective,Currency (2),Currency code (3),Currency subunit (4)
0,NaN,Afghanistan,Islamic Republic of Afghanistan,AF,Kabul,Afghan,Afghan,afghani,AFN,pul
1,NaN,Åland Islands (AX1),Åland Islands,AX,Mariehamn,Åland Islander,of the Åland Islands,euro,EUR,cent
2,NaN,Albania,Republic of Albania,AL,Tirana,Albanian,Albanian,lek,ALL,[qindar (pl. qindarka)]
3,NaN,Algeria,People’s Democratic Republic of Algeria,DZ,Algiers,Algerian,Algerian,Algerian dinar,DZD,centime
4,NaN,American Samoa (AS1),Territory of American Samoa,AS,Pago Pago (AS2),American Samoan,American Samoan,US dollar,USD,cent


In [36]:
df[0].head()

,0,1
0,1.1.2023,"Croatia, currency: ‘kuna’ replaced by ‘euro’"
1,19.12.2022,"Sierra Leone, currency code: ‘SLL’ replaced by..."
2,7.10.2022,Turkey: ‘Turkey’ replaced by ‘Türkiye’; ‘Repub...
3,6.10.2022,"Kazakhstan, capital: ‘Nur-Sultan’ replaced by ..."
4,7.2.2022,United Arab Emirates: demonym ‘Emirian’ is rep...


Use table with index of 1 and discard other data

In [37]:
df = df[1]

Inspect columns and drop ones that are not our concern

In [38]:
df.columns

Index(['Unnamed: 0', 'Short name', 'Full name', 'Country code (1)',
       'Capital/  administrative centre', 'Demonym', 'Adjective',
       'Currency (2)', 'Currency code (3)', 'Currency subunit (4)'],
      dtype='object')

In [39]:
df = df.drop([ 'Unnamed: 0','Capital/  administrative centre', 'Demonym', 'Adjective','Currency (2)', 'Currency code (3)', 'Currency subunit (4)'  ], axis=1)

In [40]:
df.head()

,Short name,Full name,Country code (1)
0,Afghanistan,Islamic Republic of Afghanistan,AF
1,Åland Islands (AX1),Åland Islands,AX
2,Albania,Republic of Albania,AL
3,Algeria,People’s Democratic Republic of Algeria,DZ
4,American Samoa (AS1),Territory of American Samoa,AS


In inport data there are some duplicated rows. Duplicate is caused by supplementary data in table

In [41]:
df = df.drop_duplicates()

Need to get rid of annotations. We will use regular expression to clean them.

In [42]:
repl = lambda m: m.group(1)
df["Short name"] = df["Short name"].str.replace("(.*)\(...\)$", repl, regex=True)

Now lets clean empty space at the end

In [43]:
repl = lambda m: m.group(1)
df["Short name"] = df["Short name"].str.replace("^(.*)?\s$", repl, regex=True)

In [44]:
df.columns

Index(['Short name', 'Full name', 'Country code (1)'], dtype='object')

We are changing column naming scheme

In [45]:
df.columns = ["Short name", "Full name", "Code"]

Folowing code is to make sure we did not lost umlaut in Turkiye

In [46]:
df[df["Code"] == "TR"]

,Short name,Full name,Code
248,Türkiye,Republic of Türkiye,TR


In [47]:
pd.set_option('display.max_rows', None)

Due to import we lost Namibia country code as it was inpterpreted as NaN. We are fixing this by filling all NA.

In [48]:
df.fillna('NA',inplace=True)

Some countries name contains duplicates in references as Salvadore. We are after countries with valida 2 letter codes.

In [49]:
df[ df["Short name"].str.contains("Salvador")]

,Short name,Full name,Code
71,El Salvador,Republic of El Salvador,SV
208,Salvador: see ‘El Salvador’,Salvador: see ‘El Salvador’,Salvador: see ‘El Salvador’


Fix it

In [50]:
df = df[ df["Code"].str.match("^\w\w$")]

In [51]:
len(df)

247

Final list of codes to be reviewed manually:

In [52]:
df

,Short name,Full name,Code
0,Afghanistan,Islamic Republic of Afghanistan,AF
1,Åland Islands,Åland Islands,AX
2,Albania,Republic of Albania,AL
3,Algeria,People’s Democratic Republic of Algeria,DZ
4,American Samoa,Territory of American Samoa,AS
5,Andorra,Principality of Andorra,AD
6,Angola,Republic of Angola,AO
7,Anguilla,Anguilla,AI
8,Antarctica,Antarctica,AQ
9,Antigua and Barbuda,Antigua and Barbuda,AG


Change colums name to align with pega

In [53]:
df.columns = ["pyLabel","FullName","pyID"]

In [55]:
df.to_csv("out.csv")

In [58]:
df[df["pyID"]=="ESP"]

,pyLabel,FullName,pyID
